In [ ]:
import sqlite3

conn = sqlite3.connect('twitter.db')
print("Successfully connected db")

In [ ]:
def create_keyword_table(db_connection: sqlite3.Connection):
    """ Create sensitive keyword table in a sqlite db

    This table stores all the labeled keywords used to perform searches on twitter.
    Most of the keywords are extracted from NSFW (Not suitable for work content).

    Keyword Categories:
    1. politically incorrect or sensitive content
    2. violence or terrorism content
    3. pornography
    4. spam
    5. normal content

    Keyword categories are mapped to their numeric labels. For example, the label of politically incorrect content is 1, and violence content has label 2, so on and so forth.
    
    Args:
        db_connection: sqlite database connection object
    """

    c = db_connection.cursor()
    c.execute(
        """
        CREATE TABLE IF NOT EXISTS KEYWORDS(
            id INT PRIMARY KEY AUTOINCREMENT NOT NULL,
            keyword TEXT NOT NULL,
            keyword_category_name CHAR(50),
            keyword_category_label INT
        );
        """
    )
    c.executescript(
        """
        CREATE INDEX keyword_index
        ON KEYWORDS (keyword, keyword_category_name);
        CREATE INDEX keyword_label_index
        ON KEYWORDS (keyword, keyword_category_label);
        """
    )


def create_post_category_table(db_connection: sqlite3.Connection):
    """ Create post category table in a sqlite db

    This table stores all the post labels.

    Post Categories:
    1. politically incorrect or sensitive content
    2. violence or terrorism content
    3. pornography
    4. spam
    5. normal content

    Keyword categories are mapped to their numeric labels. For example, the label of politically incorrect content is 1, and violence content has label 2, so on and so forth.
    
    Args:
        db_connection: sqlite database connection object
    """

    c = db_connection.cursor()
    c.execute(
        """
        CREATE TABLE IF NOT EXISTS POST_CATEGORY(
            id INT PRIMARY KEY AUTOINCREMENT NOT NULL,
            post_category_name CHAR(50),
            post_category_label INT
        );
        """
    )
    c.executescript(
        """
        CREATE INDEX post_category_index ON POST_CATEGORY (post_category_name);
        CREATE INDEX post_category_label_index ON POST_CATEGORY (post_category_label);
        """
    )






def create_twitter_page_table(db_connection: sqlite3.Connection):
    """ Create table for twitter page

    This table stores the search result in the form of html source code.
    
    Args:
        db_connection: sqlite database connection object
    """

    c = db_connection.cursor()
    c.execute(
        """
        CREATE TABLE IF NOT EXISTS TWITTER_PAGE (
            id INT PRIMARY KEY AUTOINCREMENT NOT NULL,
            keyword TEXT NOT NULL,
            page_src TEXT,
            create_dt DATETIME NOT NULL 
        );
        """
    )
    c.executescript(
        """
        CREATE INDEX twitter_page_keyword_index
        ON TWITTER_PAGE (keyword);
        """
    )


def create_twitter_post_table(db_connection: sqlite3.Connection):
    """ Create table for twitter post

    This table stores the actual twitter post which will become the training data for our text classification model.
    Keyword category and post category may not always be the same. We can count the number of mismatches between the keyword label and the post label, which then can be used to measure the weight of the keyword in illegal contents.
    
    Args:
        db_connection: sqlite database connection object
    
    """
    
    c = db_connection.cursor()
    c.execute(
        """
        CREATE TABLE IF NOT EXISTS TWITTER_POST (
            id INT PRIMARY KEY AUTOINCREMENT NOT NULL,
            keyword TEXT NOT NULL,
            keyword_category_label INT,
            post_content TEXT,
            post_by CHAR(100),
            post_dt DATETIME,
            post_category_label INT,
            create_dt DATETIME NOT NULL 
        );
        """
    )
    c.executescript(
        """
        CREATE INDEX post_keyword_index
        ON TWITTER_POST (keyword);

        CREATE INDEX post_keyword_label_index
        ON TWITTER_POST (keyword_category_label);

        CREATE INDEX post_label_index
        ON TWITTER_POST (post_category_label);
        """
    )

In [ ]:
create_keyword_table(conn)
create_post_category_table(conn)
create_twitter_page_table(conn)
create_twitter_post_table(conn)

In [ ]:
def init_post_label(conn: sqlite3.Connection, script_path: str):
    with open(script_path) as f:
        script = f.read()

    conn.executescript(script)


def init_keywords(conn: sqlite3.Connection, data_path: str, label_name: str, label_value: int):
    c = conn.cursor()
    with open(data_path) as f:
        data = [s.strip("\n") for s in f.readlines()]

    for d in data:
        sql = f"""
            INSERT into KEYWORDS (keyword, keyword_category_name, keyword_category_label)
            VALUES (?, ?, ?);
        """
        c.execute(sql, (d, label_name, label_value))

    conn.commit()
    

In [95]:
init_keywords(conn, './data/敏感词库/敏感事件.txt', 'incident', 7)

In [ ]:
c = conn.cursor()
cursor = c.execute("select * from KEYWORDS")

for row in cursor:
    print(row)

In [ ]:
c.execute("delete from KEYWORDS")
conn.commit()

In [ ]:
init_post_label(conn, './sql/init_post_label.sql')

In [ ]:
with open("cookie.txt", 'r') as f:
    cookies = [{"name":s[:s.strip().index("=")], "value":(s[s.strip().index("=")+1:].strip('\n'))} for s in f.readlines()]

cookies
# cookies = {key: value for key, value in [(s[0], s[1]) for s in  ]}

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.expected_conditions import presence_of_element_located

user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.0.3 Safari/605.1.15"
opt = webdriver.ChromeOptions()
opt.add_argument('--user-agent=%s' % user_agent)


with webdriver.Chrome("./chromedriver", options=opt) as driver:
    # configure wait
    wait = WebDriverWait(driver, 120)
    
    # visit twitter
    driver.get("https://twitter.com")
    
#     # add cookie
    for cookie in cookies:
        driver.add_cookie(cookie)
        
    driver.get("https://twitter.com/search?q=%E5%8A%A0wx&src=typed_query")
    
    # wait until contents are loaded
    first_result = wait.until(presence_of_element_located((By.XPATH, "//*[@id=\"react-root\"]/div/div/div[2]/main/div/div/div/div[1]/div/div[2]/div/div/section/div/div/div[1]/div/div/article/div/div/div/div[2]/div[2]/div[2]/div[2]/div")))
    html = driver.page_source

In [ ]:
html

In [ ]:
first_result.find_elements_by_tag_name("span")